Import potrzebnych bibliotek

In [38]:
import pandas as pd
import numpy as np

Wczyatnie danych z pliku CSV

In [39]:
plik = 'data/diabetes_prediction_dataset.csv'
df = pd.read_csv(plik)

Sprawdzenie typów danych oraz przekształcenie zmiennych kategorycznych

In [40]:
df.dtypes

gender                  object
age                    float64
hypertension             int64
heart_disease            int64
smoking_history         object
bmi                    float64
HbA1c_level            float64
blood_glucose_level      int64
diabetes                 int64
dtype: object

Wiek podany jest w pełnych latach więc zmieniamy typ danych na int

In [41]:
df['age'] = df['age'].astype(int)

Pozostałe dane które wymagają zmiany typu danych to zmienne kategoryczne - postanowiłem wykorzystać metodę one-hot encoding, w której mając kilka wartości kategorycznych stworzę tyle kolumn binarnych ile mam możliwości wartości - w tym przpadku stworzę 6 column z których tylko jedna przybierze wartość 1 a pozostałe 0 - w ten sposób przedstawię wartość kategoryczną binarnie 

In [42]:
df['smoking_history'].drop_duplicates().reset_index(drop=True)

0          never
1        No Info
2        current
3         former
4           ever
5    not current
Name: smoking_history, dtype: object

Jak widać muszę stworzyć 6 kolumn

In [43]:
df['smoking_history_never'] = np.where(df['smoking_history'] == 'never',  1, 0)
df['smoking_history_no_info'] = np.where(df['smoking_history'] == 'No Info',  1, 0)
df['smoking_history_current'] = np.where(df['smoking_history'] == 'current',  1, 0)
df['smoking_history_former'] = np.where(df['smoking_history'] == 'former',  1, 0)
df['smoking_history_ever'] = np.where(df['smoking_history'] == 'ever',  1, 0)
df['smoking_history_not_current'] = np.where(df['smoking_history'] == 'not current',  1, 0)

Teraz należy zmienić typ danych zmiennej gender

In [44]:
df['gender'].drop_duplicates().reset_index(drop=True)

0    Female
1      Male
2     Other
Name: gender, dtype: object

Jak widać mamy trzy możliwości wartości zmiennej gender - również zastosuję one-hot encoding

In [45]:
df['is_male'] = np.where(df['gender'] == 'Male', 1, 0)
df['is_female'] = np.where(df['gender'] == 'Female', 1, 0)
df['is_other'] = np.where(df['gender'] == 'Other', 1, 0)

Możemy sprawdzić skuteczność konwersji danych poprzez zrobienie testowej kolumny z sumą z kolumn konwertowanych (wynik musi zawsze wynosić 1)

In [46]:
df['smoking_test'] = df['smoking_history_current'] + df['smoking_history_former'] + df['smoking_history_ever'] + df['smoking_history_never'] + df['smoking_history_no_info'] + df['smoking_history_not_current']

df['gender_test'] = df['is_male'] + df['is_female'] + df['is_other']

In [47]:
df['smoking_test'].drop_duplicates().reset_index(drop=True)

0    1
Name: smoking_test, dtype: int32

In [48]:
df['gender_test'].drop_duplicates().reset_index(drop=True)

0    1
Name: gender_test, dtype: int32

Jak widać nie mamy wartości różnych od 1 więc poprawnie zastosowaliśmy one-hot encoding

Sprawdźmy typy danych oraz zróbmy drop niepotrzebynch już kolumn

In [50]:
df.drop(columns=['gender', 'smoking_history', 'smoking_test', 'gender_test'], inplace=True)

In [51]:
df.dtypes

age                              int32
hypertension                     int64
heart_disease                    int64
bmi                            float64
HbA1c_level                    float64
blood_glucose_level              int64
diabetes                         int64
smoking_history_never            int32
smoking_history_no_info          int32
smoking_history_current          int32
smoking_history_former           int32
smoking_history_ever             int32
smoking_history_not_current      int32
is_male                          int32
is_female                        int32
is_other                         int32
dtype: object

Następnym krokiem będzie standaryzacja danych